In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##### Importing Relevant Packages

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
import re

In [3]:
# load datasets
train = pd.read_csv("Train.csv")
test  = pd.read_csv("Test.csv")
sample  = pd.read_csv("sample_submission.csv")
var = pd.read_csv('VariableDefinitions.csv')

In [5]:
train.head(2)
train.shape

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0


(400000, 19)

##### Feature Engineering

In [6]:
useless = ['user_id', 'MRG']

In [7]:
train.REGION.fillna('REG',inplace=True)
test.REGION.fillna('REG',inplace=True)

In [8]:
train.TOP_PACK.fillna('PACK',inplace=True)
test.TOP_PACK.fillna('PACK',inplace=True)

In [9]:
minus_999999 = ['MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE','DATA_VOLUME','ON_NET','ORANGE','FREQ_TOP_PACK','TIGO','ZONE1','ZONE2']

In [10]:
for col in minus_999999:
    train[col].fillna(-99999, inplace=True)
    test[col].fillna(-99999, inplace=True)

In [11]:
train.drop(columns=[i for i in useless], inplace=True)
test.drop(columns=[i for i in useless], inplace=True)

In [12]:
data = pd.concat([train,test],axis=0)
data = data.reset_index(drop=True)

In [13]:
def tenure(x):
    'function to encode the Tenure based on the lowest amount of month spent'
    if x == 'K > 24 month':
        return 24
    elif x == 'H 15-18 month':
        return 15
    elif x == 'G 12-15 month':
        return 12
    elif x == 'J 21-24 month':
        return 21
    elif x == 'I 18-21 month':
        return 18
    elif x == 'E 6-9 month':
        return 6
    elif x == 'F 9-12 month':
        return 9
    elif x == 'D 3-6 month':
        return 3
    else:
        pass

In [14]:
data.TENURE = data.TENURE.apply(tenure)

In [15]:
categ = ['REGION','TOP_PACK'] 

In [2]:
len(data.REGION.unique()), len(data.TOP_PACK.unique())

(15, 61)

In [16]:
data = pd.get_dummies(data, prefix = categ, columns = categ)

In [17]:
data.head(3)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,TOP_PACK_VAS(IVR_Radio_Monthly),TOP_PACK_VAS(IVR_Radio_Weekly),TOP_PACK_WIFI_ Family _10MBPS,TOP_PACK_WIFI_ Family _4MBPS,TOP_PACK_WIFI_Family_2MBPS,"TOP_PACK_YMGX 100=1 hour FNF, 24H/1 month",TOP_PACK_Yewouleen_PKG,TOP_PACK_pack_chinguitel_24h,TOP_PACK_pilot_offer5,TOP_PACK_pilot_offer6
0,24,17000.0,32.0,18000.0,6000.0,34.0,-99999.0,97.0,355.0,6.0,...,0,0,0,0,0,0,0,0,0,0
1,24,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,24,1500.0,3.0,1500.0,500.0,3.0,-99999.0,30.0,30.0,-99999.0,...,0,0,0,0,0,0,0,0,0,0


##### Feature Creation

In [18]:
data['income_per_topup_amount'] = data['REVENUE'] / data['MONTANT']
data['active_per_duration'] =  data['TENURE'] / data['REGULARITY']
data['income_to_amount_ratio'] = data['REVENUE'] / data['MONTANT']
data['income_left_after_top_up'] = data['REVENUE'] - data['MONTANT']
data['tenure_to_topup_frequency'] = data['TENURE'] / data['FREQUENCE_RECH']
data['income_in90days_perRevenue'] = data['ARPU_SEGMENT'] / data['REVENUE'] 

In [19]:
train=data[data.CHURN.notnull()].reset_index(drop=True)
test=data[data.CHURN.isna()].reset_index(drop=True)

In [20]:
y = train.CHURN
train.drop('CHURN', axis=1, inplace=True)
test.drop('CHURN', axis=1, inplace=True)

In [21]:
test.head(2)
train.shape

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,TOP_PACK_Yewouleen_PKG,TOP_PACK_pack_chinguitel_24h,TOP_PACK_pilot_offer5,TOP_PACK_pilot_offer6,income_per_topup_amount,active_per_duration,income_to_amount_ratio,income_left_after_top_up,tenure_to_topup_frequency,income_in90days_perRevenue
0,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,1.0000,24.0,1.0000,0.0,-0.00024,1.0
1,24,-99999.0,-99999.0,10.0,3.0,1.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,-0.0001,12.0,-0.0001,100009.0,-0.00024,0.3


(400000, 148)

In [23]:
c = [i for i in train.columns if train[i].dtype == object]
len(c)

0

In [24]:
train.shape, y.shape, test.shape

((400000, 148), (400000,), (100000, 148))

##### NOTE
I already did feature selection and correlation to determine what features mattered in predicting customer churn for this dataset. So the clust variable below is not something I randomly chose

In [25]:
clust = [
'REVENUE',
'MONTANT',
'FREQUENCE_RECH',
'ARPU_SEGMENT',
'FREQUENCE',
'ON_NET',
'ORANGE',
'REGULARITY',
'FREQ_TOP_PACK',
'REGION_REG',
'TOP_PACK_PACK']

The cell below is quite computationally expensive and should only be run if you're interested in seeing the plots I used in determining the optimal k values, using the elbow method, also most k values i got were 2.

In [26]:
# for j in clust:
#     distortions = []
#     if train[j].dtype == object:
#         dummies = pd.get_dummies(train[j], drop_first=True)
#         for i in range(1, 11):
#             km = KMeans(
#                 n_clusters=i, init='k-means++',
#                 n_init=20, max_iter=300,
#                 tol=1e-04, random_state=0
#             )
#             km.fit(dummies)
#             distortions.append(km.inertia_)

#         # plot
#         print(j, km.inertia_)
#         plt.plot(range(1, 11), distortions, marker='o')
#         plt.xlabel('Number of clusters')
#         plt.ylabel('Distortion')
#         plt.show()
    
#     else:
#         for i in range(1, 11):
#             km = KMeans(
#                 n_clusters=i, init='k-means++',
#                 n_init=20, max_iter=300,
#                 tol=1e-04, random_state=0
#             )
#             km.fit(pd.DataFrame(train[j]))
#             distortions.append(km.inertia_)

#         # plot

#         print(j, km.inertia_)
#         plt.plot(range(1, 11), distortions, marker='o')
#         plt.xlabel('Number of clusters')
#         plt.ylabel('Distortion')
#         plt.show()

In [27]:
kmeans = KMeans(
        n_clusters=2, init='k-means++',
        n_init=20, max_iter=300,
        tol=1e-04, random_state=0
    )

In [28]:
%%capture #To avoid clumsy cell output

for col in clust:
    if train[col].dtype == object:
        dummies = pd.get_dummies(train[col], drop_first=True)
        test_dum = pd.get_dummies(test[col], drop_first=True)
        kmeans.fit(dummies)
        train[col+'_grouped'] = kmeans.labels_
        test[col+'_grouped'] = kmeans.predict(test_dum)
    else:
        kmeans.fit(pd.DataFrame(train[col]))
        train[col+'_grouped'] = kmeans.labels_
        test[col+'_grouped'] = kmeans.predict(pd.DataFrame(test[col]))

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

...

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

KMeans(n_clusters=2, n_init=20, random_state=0)

In [29]:
train.shape, test.shape

((400000, 159), (100000, 159))

In [38]:
del data

In [40]:
sum(train.isna().sum())

0

The cell below is quite computationally expensive and should only be run if you're interested in seeing the plots I used in determining the optimal k values, using the elbow method, also most k values i got were 2.

In [3]:
# distortions = []
# for i in range(1, 11):
#     km = KMeans(
#         n_clusters=i, init='k-means++',
#         n_init=20, max_iter=300,
#         tol=1e-04, random_state=0
#     )
#     km.fit(pd.DataFrame(train))
#     distortions.append(km.inertia_)

# # plot
# plt.plot(range(1, 11), distortions, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Distortion')
# plt.show()

In [42]:
km = KMeans(
        n_clusters=2, init='k-means++',
        n_init=20, max_iter=300,
        tol=1e-04, random_state=0
    )

In [43]:
km.fit(train)

KMeans(n_clusters=2, n_init=20, random_state=0)

In [44]:
train['groups'] = km.labels_
test['groups'] = km.predict(test)

In [45]:
train.shape, y.shape, test.shape

((400000, 160), (400000,), (100000, 160))

In [46]:
train.head(3)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,FREQUENCE_RECH_grouped,ARPU_SEGMENT_grouped,FREQUENCE_grouped,ON_NET_grouped,ORANGE_grouped,REGULARITY_grouped,FREQ_TOP_PACK_grouped,REGION_REG_grouped,TOP_PACK_PACK_grouped,groups
0,24,17000.0,32.0,18000.0,6000.0,34.0,-99999.0,97.0,355.0,6.0,...,1,1,0,0,0,1,0,1,1,0
1,24,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,...,1,1,0,0,0,1,0,0,1,0
2,24,1500.0,3.0,1500.0,500.0,3.0,-99999.0,30.0,30.0,-99999.0,...,1,1,0,0,0,1,0,1,1,0


##### Local Validation

In [47]:
X_1, X_2, y_1, y_2 = train_test_split(train, y, test_size = 0.01, random_state = 0)

In [48]:
X_3, X_4, y_3, y_4 = train_test_split(X_2, y_2, test_size = 0.1, random_state = 0)

In [49]:
X_3.shape, y_3.shape

((3600, 160), (3600,))

In [ ]:
val_model = CatBoostClassifier(random_seed = 10, n_estimators = 1000)

In [ ]:
val_model.fit(X_3, y_3)

In [ ]:
val_pred = val_model.predict_proba(X_4)[:, 1]

In [80]:
log_loss(y_4, val_pred)

0.2631137325635713

In [52]:
log_loss(y_4, val_pred)

0.26259819218504604

In [66]:
log_loss(y_4, val_pred)

0.2568614664247297

In [ ]:
log_loss(y_4, val_pred)

Local CV log_loss score of about 0.255 seems okay, but caution is that it's just a very sample from our dataset.

##### Model Training

- Models I used in making my preddictions are catboost and xgboost

In [50]:
train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x), inplace=True)
test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x), inplace=True)

### I'm renaming the columns because Lightgbm(LGBM) model would give an error saying it can't accept JSON as column names.

In [84]:
train.head(2)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,FREQUENCE_RECH_grouped,ARPU_SEGMENT_grouped,FREQUENCE_grouped,ON_NET_grouped,ORANGE_grouped,REGULARITY_grouped,FREQ_TOP_PACK_grouped,REGION_REG_grouped,TOP_PACK_PACK_grouped,groups
0,24,17000.0,32.0,18000.0,6000.0,34.0,-99999.0,97.0,355.0,6.0,...,1,1,0,0,0,1,0,1,1,0
1,24,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,...,1,1,0,0,0,1,0,0,1,0


25-KFold Catboost model

In [85]:
train.shape, test.shape, y.shape

((400000, 160), (100000, 160), (400000,))

In [88]:
errcb1=[]
y_pred_totcb1=[]
fold=KFold(n_splits=25)#25
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m1  = CatBoostClassifier(n_estimators=1000,eval_metric='Logloss',random_seed= 10, use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:,1]
    print("err: ",log_loss(y_test,preds))
    errcb1.append((log_loss(y_test,preds)))
    p1 = m1.predict_proba(test)[:,1]
    y_pred_totcb1.append(p1)
    i+=1
np.mean(errcb1)

1 iter
Learning rate set to 0.137797
0:	learn: 0.5031452	test: 0.5031452	test1: 0.5022872	best: 0.5022872 (0)	total: 172ms	remaining: 2m 51s
100:	learn: 0.2509054	test: 0.2509054	test1: 0.2480670	best: 0.2480670 (100)	total: 16.3s	remaining: 2m 25s
200:	learn: 0.2491595	test: 0.2491595	test1: 0.2477578	best: 0.2477556 (194)	total: 30.8s	remaining: 2m 2s
300:	learn: 0.2477787	test: 0.2477787	test1: 0.2478270	best: 0.2476115 (216)	total: 50.5s	remaining: 1m 57s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2476115411
bestIteration = 216

Shrink model to first 217 iterations.


err:  0.24761154110800296
2 iter
Learning rate set to 0.137797
0:	learn: 0.5115097	test: 0.5115097	test1: 0.5107237	best: 0.5107237 (0)	total: 223ms	remaining: 3m 42s
100:	learn: 0.2510100	test: 0.2510100	test1: 0.2492923	best: 0.2492473 (86)	total: 23.5s	remaining: 3m 29s
200:	learn: 0.2491564	test: 0.2491564	test1: 0.2490761	best: 0.2490456 (195)	total: 42s	remaining: 2m 46s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2490455764
bestIteration = 195

Shrink model to first 196 iterations.


err:  0.24904557642250227
3 iter
Learning rate set to 0.137797
0:	learn: 0.5130667	test: 0.5130667	test1: 0.5122895	best: 0.5122895 (0)	total: 483ms	remaining: 8m 2s
100:	learn: 0.2509320	test: 0.2509320	test1: 0.2469596	best: 0.2469596 (100)	total: 19.5s	remaining: 2m 53s
200:	learn: 0.2492311	test: 0.2492311	test1: 0.2465169	best: 0.2464827 (189)	total: 35.8s	remaining: 2m 22s
300:	learn: 0.2478480	test: 0.2478480	test1: 0.2463364	best: 0.2463364 (300)	total: 54.2s	remaining: 2m 5s
400:	learn: 0.2466516	test: 0.2466516	test1: 0.2464571	best: 0.2463364 (300)	total: 1m 12s	remaining: 1m 47s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2463363593
bestIteration = 300

Shrink model to first 301 iterations.


err:  0.24633635927157851
4 iter
Learning rate set to 0.137797
0:	learn: 0.5131187	test: 0.5131187	test1: 0.5139142	best: 0.5139142 (0)	total: 176ms	remaining: 2m 55s
100:	learn: 0.2505563	test: 0.2505563	test1: 0.2542533	best: 0.2542533 (100)	total: 17.8s	remaining: 2m 38s
200:	learn: 0.2489606	test: 0.2489606	test1: 0.2538519	best: 0.2538446 (189)	total: 33.6s	remaining: 2m 13s
300:	learn: 0.2476394	test: 0.2476394	test1: 0.2537160	best: 0.2536825 (284)	total: 49.7s	remaining: 1m 55s
400:	learn: 0.2465250	test: 0.2465250	test1: 0.2535437	best: 0.2535437 (400)	total: 1m 5s	remaining: 1m 37s
500:	learn: 0.2455330	test: 0.2455330	test1: 0.2535398	best: 0.2534809 (450)	total: 1m 21s	remaining: 1m 20s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.253480914
bestIteration = 450

Shrink model to first 451 iterations.


err:  0.25348091398596706
5 iter
Learning rate set to 0.137797
0:	learn: 0.5133129	test: 0.5133129	test1: 0.5137363	best: 0.5137363 (0)	total: 208ms	remaining: 3m 27s
100:	learn: 0.2505739	test: 0.2505739	test1: 0.2563574	best: 0.2563537 (98)	total: 19.3s	remaining: 2m 52s
200:	learn: 0.2488776	test: 0.2488776	test1: 0.2559505	best: 0.2559362 (197)	total: 35.1s	remaining: 2m 19s
300:	learn: 0.2475273	test: 0.2475273	test1: 0.2558050	best: 0.2557572 (281)	total: 51s	remaining: 1m 58s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2557571766
bestIteration = 281

Shrink model to first 282 iterations.


...

err:  0.24718674065748922
23 iter
Learning rate set to 0.137797
0:	learn: 0.5128012	test: 0.5128012	test1: 0.5125624	best: 0.5125624 (0)	total: 162ms	remaining: 2m 41s
100:	learn: 0.2505028	test: 0.2505028	test1: 0.2574160	best: 0.2574160 (100)	total: 17.9s	remaining: 2m 38s
200:	learn: 0.2488369	test: 0.2488369	test1: 0.2572106	best: 0.2571228 (155)	total: 34.2s	remaining: 2m 15s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2571227547
bestIteration = 155

Shrink model to first 156 iterations.


err:  0.2571227546896069
24 iter
Learning rate set to 0.137797
0:	learn: 0.5125811	test: 0.5125811	test1: 0.5137962	best: 0.5137962 (0)	total: 180ms	remaining: 3m
100:	learn: 0.2506285	test: 0.2506285	test1: 0.2559610	best: 0.2559610 (100)	total: 19.7s	remaining: 2m 55s
200:	learn: 0.2488817	test: 0.2488817	test1: 0.2560270	best: 0.2559447 (103)	total: 37.6s	remaining: 2m 29s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2559447335
bestIteration = 103

Shrink model to first 104 iterations.


err:  0.2559447334508798
25 iter
Learning rate set to 0.137797
0:	learn: 0.5127269	test: 0.5127269	test1: 0.5108666	best: 0.5108666 (0)	total: 242ms	remaining: 4m 1s
100:	learn: 0.2508848	test: 0.2508848	test1: 0.2483940	best: 0.2483940 (100)	total: 20.9s	remaining: 3m 6s
200:	learn: 0.2493418	test: 0.2493418	test1: 0.2482886	best: 0.2482032 (146)	total: 37s	remaining: 2m 27s
300:	learn: 0.2478151	test: 0.2478151	test1: 0.2481083	best: 0.2480436 (277)	total: 53.6s	remaining: 2m 4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2480436365
bestIteration = 277

Shrink model to first 278 iterations.


err:  0.24804363650294955


0.2519568953327828

In [89]:
cat_sub = sample.copy()

In [91]:
cat_sub.CHURN = np.mean(y_pred_totcb1, axis=0)

In [92]:
cat_sub.to_csv('25KFold_Cat.csv', index = False)

3-StratKFold Catboost model

In [93]:
train.shape, test.shape, y.shape

((400000, 160), (100000, 160), (400000,))

In [63]:
sc = StandardScaler()

In [105]:
errcb2=[]
y_pred_totcb2=[]
fold = StratifiedKFold(n_splits=3, shuffle= True, random_state= 0)#5
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_pred = sc.transform(test)
    
    m2 = CatBoostClassifier(n_estimators=500,eval_metric='Logloss',random_seed= 10, use_best_model=True)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=300,verbose=100)
    preds=m2.predict_proba(X_test)[:,1]
    print("err: ",log_loss(y_test,preds))
    errcb2.append((log_loss(y_test,preds)))
    p2 = m2.predict_proba(X_pred)[:,1]
    y_pred_totcb2.append(p2)
    i+=1
np.mean(errcb2)

1 iter
Learning rate set to 0.170244
0:	learn: 0.4812967	test: 0.4812967	test1: 0.4806269	best: 0.4806269 (0)	total: 117ms	remaining: 58.2s
100:	learn: 0.2494701	test: 0.2494701	test1: 0.2528859	best: 0.2528859 (100)	total: 12.6s	remaining: 49.8s
200:	learn: 0.2471093	test: 0.2471093	test1: 0.2528639	best: 0.2528156 (154)	total: 25s	remaining: 37.1s
300:	learn: 0.2450455	test: 0.2450455	test1: 0.2529669	best: 0.2528156 (154)	total: 35.9s	remaining: 23.8s
400:	learn: 0.2430554	test: 0.2430554	test1: 0.2531818	best: 0.2528156 (154)	total: 47.9s	remaining: 11.8s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2528155906
bestIteration = 154

Shrink model to first 155 iterations.


err:  0.2528155906037959
2 iter
Learning rate set to 0.170244
0:	learn: 0.4784894	test: 0.4784894	test1: 0.4789028	best: 0.4789028 (0)	total: 264ms	remaining: 2m 11s
100:	learn: 0.2498448	test: 0.2498448	test1: 0.2522361	best: 0.2522234 (98)	total: 12.9s	remaining: 51.1s
200:	learn: 0.2475281	test: 0.2475281	test1: 0.2521749	best: 0.2521719 (147)	total: 25.5s	remaining: 38s
300:	learn: 0.2456093	test: 0.2456093	test1: 0.2522623	best: 0.2521564 (239)	total: 39.5s	remaining: 26.1s
400:	learn: 0.2437155	test: 0.2437155	test1: 0.2525340	best: 0.2521564 (239)	total: 51.2s	remaining: 12.6s
499:	learn: 0.2421210	test: 0.2421210	test1: 0.2527871	best: 0.2521564 (239)	total: 1m 2s	remaining: 0us

bestTest = 0.2521564189
bestIteration = 239

Shrink model to first 240 iterations.


err:  0.2521564188803394
3 iter
Learning rate set to 0.170244
0:	learn: 0.4718606	test: 0.4718606	test1: 0.4721915	best: 0.4721915 (0)	total: 151ms	remaining: 1m 15s
100:	learn: 0.2499391	test: 0.2499391	test1: 0.2522131	best: 0.2522131 (100)	total: 13.1s	remaining: 51.7s
200:	learn: 0.2477287	test: 0.2477287	test1: 0.2520730	best: 0.2520730 (200)	total: 25.3s	remaining: 37.6s
300:	learn: 0.2456626	test: 0.2456626	test1: 0.2522210	best: 0.2520457 (211)	total: 37.1s	remaining: 24.5s
400:	learn: 0.2437305	test: 0.2437305	test1: 0.2524362	best: 0.2520457 (211)	total: 49s	remaining: 12.1s
499:	learn: 0.2420665	test: 0.2420665	test1: 0.2526516	best: 0.2520457 (211)	total: 1m 1s	remaining: 0us

bestTest = 0.2520456985
bestIteration = 211

Shrink model to first 212 iterations.


err:  0.2520456984589125


0.25233923598101593

In [106]:
cat_1_sub = sample.copy()

In [107]:
cat_1_sub.CHURN = np.mean(y_pred_totcb2, axis=0)

In [108]:
#cat_1_sub.to_csv('3STRATKFold_Cat.csv', index = False) # early 100

In [109]:
cat_1_sub.to_csv('3STRATKFold_Cat3scaled.csv', index = False)# early 300

25-KFold Xgbboost

In [92]:
errcb3=[]
y_pred_totcb3=[]
fold=KFold(n_splits=25)#25
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m3  = XGBClassifier(n_estimators=1000,random_seed = 10, use_best_model=True, eval_metric = 'logloss')
    m3.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m3.predict_proba(X_test)[:,1]
    print("err: ",log_loss(y_test,preds))
    errcb3.append((log_loss(y_test,preds)))
    p3 = m3.predict_proba(test)[:,1]
    y_pred_totcb3.append(p3)
    i+=1
np.mean(errcb3)

1 iter
[0]	validation_0-logloss:0.52151	validation_1-logloss:0.52123
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.24283	validation_1-logloss:0.24865
Stopping. Best iteration:
[53]	validation_0-logloss:0.24612	validation_1-logloss:0.24774



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.2477421123352524
2 iter
[0]	validation_0-logloss:0.52149	validation_1-logloss:0.52101
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.24201	validation_1-logloss:0.25107
Stopping. Best iteration:
[21]	validation_0-logloss:0.24913	validation_1-logloss:0.25018



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.25017494368712506
3 iter
[0]	validation_0-logloss:0.52156	validation_1-logloss:0.52068
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.24290	validation_1-logloss:0.24705
Stopping. Best iteration:
[70]	validation_0-logloss:0.24502	validation_1-logloss:0.24694



...

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.2562502689226876
25 iter
[0]	validation_0-logloss:0.52160	validation_1-logloss:0.52038
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.24257	validation_1-logloss:0.24899
Stopping. Best iteration:
[54]	validation_0-logloss:0.24593	validation_1-logloss:0.24850



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.24849955794200967


0.2524541595091435

In [93]:
xgb_sub = sample.copy()

In [94]:
xgb_sub.CHURN = np.mean(y_pred_totcb3, axis=0)

In [95]:
xgb_sub.to_csv('25KFold_Xgb.csv', index = False)

3-StratKFold Xgbboost model

In [55]:
train.shape, test.shape, y.shape

((400000, 160), (100000, 160), (400000,))

In [64]:
errcb4=[]
y_pred_totcb4=[]
fold = StratifiedKFold(n_splits=3, shuffle= True, random_state= 0)#5
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_pred = sc.transform(test)
    
    m4 = XGBClassifier(n_estimators=1000,eval_metric='logloss',random_seed= 10, use_best_model=True)
    m4.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m4.predict_proba(X_test)[:,1]
    print("err: ",log_loss(y_test,preds))
    errcb4.append((log_loss(y_test,preds)))
    p4 = m4.predict_proba(X_pred)[:,1]
    y_pred_totcb4.append(p4)
    i+=1
np.mean(errcb4)

1 iter
[0]	validation_0-logloss:0.52145	validation_1-logloss:0.52172
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.23891	validation_1-logloss:0.25469
Stopping. Best iteration:
[26]	validation_0-logloss:0.24664	validation_1-logloss:0.25354



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.25353394341942115
2 iter
[0]	validation_0-logloss:0.52148	validation_1-logloss:0.52206
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.23955	validation_1-logloss:0.25370
Stopping. Best iteration:
[31]	validation_0-logloss:0.24681	validation_1-logloss:0.25280



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.2528070067666714
3 iter
[0]	validation_0-logloss:0.52162	validation_1-logloss:0.52193
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.24012	validation_1-logloss:0.25353
Stopping. Best iteration:
[28]	validation_0-logloss:0.24707	validation_1-logloss:0.25244



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, random_seed=10, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              use_best_model=True, validate_parameters=False, verbosity=None)

err:  0.25243592398959064


0.25292562472522767

In [65]:
xgb_1_sub = sample.copy()

In [66]:
xgb_1_sub.CHURN = np.mean(y_pred_totcb4, axis=0)

In [67]:
xgb_1_sub.to_csv('3STRATKFold_Xgbscaled.csv', index = False)

##### Blending of  predictions

In [103]:
best = pd.read_csv('25KFold_Cat.csv')
sec = pd.read_csv('25KFold_Xgb.csv')

In [104]:
blend = ((sec.CHURN*0.4) + (best.CHURN*0.6))*0.95 + (best.CHURN*0.05)

In [105]:
blend_sub = sample.copy()

In [106]:
blend_sub.CHURN = blend

In [107]:
blend_sub.to_csv('XGB_CAT_blend3.csv', index = False)

In [108]:
best.head(3)

,user_id,CHURN
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,0.783101
1,5335efd940280b82143272275637d1e65d37eadb,0.639718
2,a581f4fa08677c26f83f643248c667e241043086,0.133376


In [109]:
blend_sub.head(3)

,user_id,CHURN
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,0.781986
1,5335efd940280b82143272275637d1e65d37eadb,0.646253
2,a581f4fa08677c26f83f643248c667e241043086,0.127451
